In [1]:
# for "2. Data Loading"
import pandas as pd

# for "3-1. Feature Generation"
import numpy as np

# for "3-2. Feature Engineering"
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import RobustScaler, StandardScaler

# for "4. Modeling with Pycaret"
from pycaret.regression import *

# for "5. Modeling with CatBoostRegressor"
from catboost import CatBoostRegressor
import optuna
from optuna import Trial
from optuna.samplers import TPESampler
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split, StratifiedKFold

### 1121
추가 사항 석식계 예측에 자기계발 열을 제거하여 학습 진행

### 1122
추가 사항 독립변수 스케일링 진행

In [2]:
# 데이터 로드
pre_tr = pd.read_csv('data_preprocess/pre_tr_1118.csv',encoding = "cp949")
pre_te = pd.read_csv('data_preprocess/pre_te_1118.csv',encoding = "cp949")
pre_tr = pre_tr.set_index("일자")
pre_te = pre_te.set_index("일자")
today = "1122_1"

In [3]:
# 데이터 확인
pre_tr.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1205 entries, 2016-02-01 to 2021-01-26
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   요일      1205 non-null   int64  
 1   정원수     1205 non-null   int64  
 2   휴가자수    1205 non-null   int64  
 3   출장자수    1205 non-null   int64  
 4   야근자수    1205 non-null   int64  
 5   재택근무자수  1205 non-null   float64
 6   조식메뉴    1205 non-null   object 
 7   중식메뉴    1205 non-null   object 
 8   석식메뉴    1205 non-null   object 
 9   중식계     1205 non-null   float64
 10  석식계     1205 non-null   float64
 11  출근자수    1205 non-null   float64
 12  월       1205 non-null   int64  
 13  년도      1205 non-null   int64  
dtypes: float64(4), int64(7), object(3)
memory usage: 141.2+ KB


조식메뉴, 중식메뉴, 석식메뉴 다 없어도 될거 같은데,,,?
일자도 일단 제외하고 진행

In [4]:
#중식 타겟 데이터
y_lun = pre_tr["중식계"]
# 석식 타겟 데이터
y_din = pre_tr["석식계"]

## 중식계 예측

In [5]:
train_dr = pre_tr.loc[:,"조식메뉴":"석식계"]
X_train = pre_tr.drop(train_dr,axis=1)

In [6]:
# minmax scale 활용
from sklearn.preprocessing import MinMaxScaler
minmax = MinMaxScaler()
col_na = X_train.loc[:, "정원수":"출근자수"].columns
col_na
X_train.loc[:, "정원수":"출근자수"] = minmax.fit_transform(
    X_train[col_na])

In [7]:
X_train["중식계"] = pre_tr["중식계"]
display(X_train)

,요일,정원수,휴가자수,출장자수,야근자수,재택근무자수,출근자수,월,년도,중식계
일자,,,,,,,,,,
2016-02-01,1,0.000000,0.022481,0.323442,0.227969,0.000000,0.664300,2,2016,1039.0
2016-02-02,2,0.000000,0.022481,0.391691,0.305556,0.000000,0.649451,2,2016,867.0
2016-02-03,3,0.000000,0.027477,0.412463,0.106322,0.000000,0.641059,2,2016,1017.0
2016-02-04,4,0.000000,0.067444,0.531157,0.340038,0.000000,0.584248,2,2016,978.0
2016-02-05,5,0.000000,0.212323,0.415430,0.032567,0.000000,0.497095,2,2016,925.0
...,...,...,...,...,...,...,...,...,...,...
2021-01-20,3,0.542614,0.043297,0.465875,0.003831,0.733583,0.611362,1,2021,1093.0
2021-01-21,4,0.542614,0.057452,0.563798,0.442529,0.658537,0.604906,1,2021,832.0
2021-01-22,5,0.542614,0.193172,0.614243,0.000958,0.568480,0.519690,1,2021,579.0


In [8]:
reg = setup(X_train, 
            preprocess = False, # True로 설정되면, 자체적인 Feature Engineering을 추가로 진행해 Predict가 불가능해진다.
            train_size = 0.999,  # 우리는 전체 데이터를 학습해 test를 예측하는게 목표이기 때문에, 0.999로 설정한다.
            target = '중식계', # 목표 변수는 중식계 이다.
            silent = True, # 엔터를 누르기 귀찮다. 궁금하면 풀어보세요
            use_gpu = False, # GPU가 있으면 사용하세요 (Cat BOost 속도 향상)
            numeric_features=list(X_train.drop(columns = ['중식계']).columns), # 모든 변수가 숫자로써의 의미가 있다.
            session_id = 2021,
            fold_shuffle = True
            )

,Description,Value
0,session_id,2021
1,Target,중식계
2,Original Data,"(1205, 10)"
3,Missing Values,False
4,Numeric Features,9
5,Categorical Features,0
6,Transformed Train Set,"(1203, 9)"
7,Transformed Test Set,"(2, 9)"
8,Shuffle Train-Test,True
9,Stratify Train-Test,False


In [9]:
top5 = compare_models(n_select = 5, sort = 'MAE')

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,64.8464,7581.0875,86.6848,0.8243,0.1060,0.0781,0.5510
gbr,Gradient Boosting Regressor,66.4847,8145.4710,89.7528,0.8109,0.1102,0.0806,0.0330
et,Extra Trees Regressor,69.8119,9094.6202,94.7995,0.7888,0.1174,0.0850,0.0700
lightgbm,Light Gradient Boosting Machine,70.1996,8766.0373,93.0467,0.7959,0.1144,0.0850,0.1300
rf,Random Forest Regressor,70.4995,8979.5624,94.2141,0.7912,0.1164,0.0857,0.0900
xgboost,Extreme Gradient Boosting,72.7878,9004.7620,94.5227,0.7905,0.1160,0.0878,0.3140
ada,AdaBoost Regressor,84.2015,11624.2774,107.5933,0.7296,0.1328,0.1038,0.0290
ridge,Ridge Regression,86.6336,12772.8044,112.5937,0.7052,0.1357,0.1037,0.0050
br,Bayesian Ridge,86.6820,12721.3154,112.3577,0.7064,0.1354,0.1036,0.0050
lr,Linear Regression,86.7157,12719.2156,112.3469,0.7065,0.1354,0.1036,0.4780


In [10]:
models = []
for m in top5:
    models.append(tune_model(m, 
                             optimize = 'MAE', 
                             choose_better = True,
                            n_iter = 30))

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,78.9316,11022.2324,104.9868,0.7111,0.1264,0.0963
1,63.0248,6526.6583,80.7877,0.8676,0.1049,0.0799
2,68.2797,7893.8903,88.8476,0.8364,0.1225,0.0888
3,71.7590,9506.3183,97.5004,0.7822,0.1135,0.0837
4,78.1836,10873.8285,104.2777,0.7404,0.1305,0.0957
5,73.5441,9811.6020,99.0535,0.7934,0.1199,0.0866
6,61.1827,6171.5282,78.5591,0.8392,0.0984,0.0762
7,66.4182,7419.4827,86.1364,0.8212,0.1025,0.0776
8,64.5398,7572.1581,87.0181,0.8072,0.1055,0.0781
9,72.7580,9991.0270,99.9551,0.7881,0.1273,0.0904


In [11]:
test_dr = pre_te.loc[:,"조식메뉴":"석식메뉴"]
X_test = pre_te.drop(test_dr,axis=1)
X_test.loc[:, "정원수":"출근자수"] = minmax.transform(
    X_test[col_na])
display(X_test)

,요일,정원수,휴가자수,출장자수,야근자수,재택근무자수,출근자수,월,년도
일자,,,,,,,,,
2021-01-27,3,0.542614,0.054122,0.418398,0.004789,0.671670,0.664300,1,2021
2021-01-28,4,0.542614,0.067444,0.507418,0.391762,0.652908,0.649451,1,2021
2021-01-29,5,0.542614,0.205662,0.617211,0.000000,0.551595,0.641059,1,2021
2021-02-01,1,0.458807,0.070774,0.335312,0.515326,0.604128,0.584248,2,2021
2021-02-02,2,0.458807,0.032473,0.430267,0.435824,0.589118,0.497095,2,2021
2021-02-03,3,0.458807,0.029975,0.468843,0.004789,0.536585,0.453841,2,2021
2021-02-04,4,0.458807,0.031640,0.504451,0.455939,0.540338,0.441575,2,2021
2021-02-05,5,0.458807,0.121565,0.626113,0.000000,0.480300,0.605552,2,2021
2021-02-08,1,0.458807,0.054122,0.394659,0.660920,0.617261,0.594577,2,2021


In [12]:
voting = blend_models(models, optimize = 'MAE')
voting = tune_model(voting, 
                 optimize = 'MAE', 
                 choose_better = True,
                 n_iter = 30)

voting = finalize_model(voting)
sample = pd.read_csv('sub/sample_submission.csv', encoding = "cp949")
layer1_pred = voting.predict(X_test)
sample['중식계'] = layer1_pred

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,72.8373,10266.7072,101.3248,0.7309,0.1202,0.0880
1,59.9027,6151.8325,78.4336,0.8752,0.1049,0.0768
2,61.6999,6522.8181,80.7640,0.8648,0.1122,0.0799
3,64.6990,7853.7951,88.6216,0.8201,0.1035,0.0757
4,70.3115,9156.4630,95.6894,0.7814,0.1180,0.0859
5,68.1600,8496.3963,92.1759,0.8211,0.1123,0.0804
6,58.1674,5536.2556,74.4060,0.8557,0.0954,0.0725
7,62.5579,6495.5110,80.5947,0.8435,0.0929,0.0716
8,58.3188,6266.5773,79.1617,0.8405,0.0921,0.0688
9,67.1869,8993.4589,94.8338,0.8092,0.1210,0.0837


## 석식계 예측

In [15]:
train_dr = pre_tr.loc[:,"조식메뉴":"석식계"]
X_train = pre_tr.drop(train_dr,axis=1)
X_train.loc[:, "정원수":"출근자수"] = minmax.fit_transform(
    X_train[col_na])
X_train["석식계"] = pre_tr["석식계"]
display(X_train)

,요일,정원수,휴가자수,출장자수,야근자수,재택근무자수,출근자수,월,년도,석식계
일자,,,,,,,,,,
2016-02-01,1,0.000000,0.022481,0.323442,0.227969,0.000000,0.664300,2,2016,331.0
2016-02-02,2,0.000000,0.022481,0.391691,0.305556,0.000000,0.649451,2,2016,560.0
2016-02-03,3,0.000000,0.027477,0.412463,0.106322,0.000000,0.641059,2,2016,573.0
2016-02-04,4,0.000000,0.067444,0.531157,0.340038,0.000000,0.584248,2,2016,525.0
2016-02-05,5,0.000000,0.212323,0.415430,0.032567,0.000000,0.497095,2,2016,330.0
...,...,...,...,...,...,...,...,...,...,...
2021-01-20,3,0.542614,0.043297,0.465875,0.003831,0.733583,0.611362,1,2021,421.0
2021-01-21,4,0.542614,0.057452,0.563798,0.442529,0.658537,0.604906,1,2021,353.0
2021-01-22,5,0.542614,0.193172,0.614243,0.000958,0.568480,0.519690,1,2021,217.0


In [16]:
# 자기계발 날 제거 
# (석식계 0인 이상치가 발견되는 열들 제거 하여 학습 데이터 생성) 
# 총 43개 열 제거
day_of_dinner0 = X_train.loc[X_train["석식계"] == 0]
display(day_of_dinner0)

,요일,정원수,휴가자수,출장자수,야근자수,재택근무자수,출근자수,월,년도,석식계
일자,,,,,,,,,,
2016-11-30,3,0.125000,0.037469,0.492582,0.000000,0.000000,0.672692,11,2016,0.0
2016-12-28,3,0.147727,0.119067,0.545994,0.000000,0.000000,0.608134,12,2016,0.0
2017-01-25,3,0.136364,0.046628,0.480712,0.000000,0.000000,0.673338,1,2017,0.0
2017-02-22,3,0.044034,0.043297,0.626113,0.000000,0.000000,0.602324,2,2017,0.0
2017-03-22,3,0.036932,0.024979,0.575668,0.000000,0.000000,0.624274,3,2017,0.0
2017-04-26,3,0.035511,0.018318,0.780415,0.000000,0.000000,0.584248,4,2017,0.0
2017-05-31,3,0.051136,0.016653,0.664688,0.000000,0.000000,0.617818,5,2017,0.0
2017-06-28,3,0.066761,0.029142,0.646884,0.000000,0.000000,0.619109,6,2017,0.0
2017-07-26,3,0.338068,0.192340,0.608309,0.000000,0.000000,0.624274,7,2017,0.0


In [17]:
# 자기계발 날 제거 
# (석식계 0인 이상치가 발견되는 열들 제거 하여 학습 데이터 생성) 
# 총 43개 열 제거
X_train = X_train.drop(day_of_dinner0.index,axis=0)
display(X_train)

,요일,정원수,휴가자수,출장자수,야근자수,재택근무자수,출근자수,월,년도,석식계
일자,,,,,,,,,,
2016-02-01,1,0.000000,0.022481,0.323442,0.227969,0.000000,0.664300,2,2016,331.0
2016-02-02,2,0.000000,0.022481,0.391691,0.305556,0.000000,0.649451,2,2016,560.0
2016-02-03,3,0.000000,0.027477,0.412463,0.106322,0.000000,0.641059,2,2016,573.0
2016-02-04,4,0.000000,0.067444,0.531157,0.340038,0.000000,0.584248,2,2016,525.0
2016-02-05,5,0.000000,0.212323,0.415430,0.032567,0.000000,0.497095,2,2016,330.0
...,...,...,...,...,...,...,...,...,...,...
2021-01-20,3,0.542614,0.043297,0.465875,0.003831,0.733583,0.611362,1,2021,421.0
2021-01-21,4,0.542614,0.057452,0.563798,0.442529,0.658537,0.604906,1,2021,353.0
2021-01-22,5,0.542614,0.193172,0.614243,0.000958,0.568480,0.519690,1,2021,217.0


In [18]:
reg = setup(X_train, 
            preprocess = False, # True로 설정되면, 자체적인 Feature Engineering을 추가로 진행해 Predict가 불가능해진다.
            train_size = 0.999,  # 우리는 전체 데이터를 학습해 test를 예측하는게 목표이기 때문에, 0.999로 설정한다.
            target = '석식계', # 목표 변수는 석식계 이다.
            silent = True, # 엔터를 누르기 귀찮다. 궁금하면 풀어보세요
            use_gpu = False, # GPU가 있으면 사용하세요 (Cat BOost 속도 향상)
            numeric_features=list(X_train.drop(columns = ['석식계']).columns), # 모든 변수가 숫자로써의 의미가 있다.
            session_id = 2021,
            fold_shuffle = True
            )

,Description,Value
0,session_id,2021
1,Target,석식계
2,Original Data,"(1162, 10)"
3,Missing Values,False
4,Numeric Features,9
5,Categorical Features,0
6,Transformed Train Set,"(1160, 9)"
7,Transformed Test Set,"(2, 9)"
8,Shuffle Train-Test,True
9,Stratify Train-Test,False


In [19]:
top5 = compare_models(n_select = 5, sort = 'MAE')

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,43.4490,3449.0813,58.3909,0.7042,0.1428,0.1035,0.5820
gbr,Gradient Boosting Regressor,44.7451,3643.6641,59.9276,0.6893,0.1420,0.1046,0.0400
lightgbm,Light Gradient Boosting Machine,45.5691,3674.3429,60.3644,0.6842,0.1453,0.1073,0.1310
et,Extra Trees Regressor,45.5735,3888.5400,61.6965,0.6672,0.1486,0.1080,0.0690
rf,Random Forest Regressor,46.1706,3868.4007,61.6473,0.6688,0.1480,0.1089,0.0900
xgboost,Extreme Gradient Boosting,47.5611,4117.1006,63.7517,0.6481,0.1519,0.1110,0.5010
ridge,Ridge Regression,53.2360,5020.5172,70.4517,0.5711,0.1698,0.1274,0.0080
br,Bayesian Ridge,53.2665,5016.4517,70.4218,0.5714,0.1693,0.1273,0.0050
lar,Least Angle Regression,53.2876,5016.1825,70.4193,0.5714,0.1691,0.1272,0.0050
lr,Linear Regression,53.2877,5016.1913,70.4194,0.5714,0.1691,0.1272,0.5990


In [20]:
models = []
for m in top5:
    models.append(tune_model(m, 
                             optimize = 'MAE', 
                             choose_better = True,
                            n_iter = 30))

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,46.8484,4418.8727,66.4746,0.6344,0.1768,0.1206
1,50.8883,5259.0214,72.5191,0.5015,0.1398,0.1023
2,51.2322,4216.0117,64.9308,0.6139,0.1778,0.1325
3,48.4042,3961.0107,62.9366,0.6551,0.1500,0.1139
4,50.9336,5644.8179,75.1320,0.5933,0.1882,0.1290
5,43.4779,3229.0708,56.8249,0.7303,0.1343,0.1019
6,43.2134,3243.8644,56.9549,0.7238,0.1446,0.1062
7,43.9392,3401.9131,58.3259,0.7400,0.1625,0.1148
8,40.0843,2648.5008,51.4636,0.6822,0.1121,0.0886
9,50.4978,4971.2633,70.5072,0.6283,0.1814,0.1280


In [21]:
test_dr = pre_te.loc[:,"조식메뉴":"석식메뉴"]
X_test = pre_te.drop(test_dr,axis=1)
X_test.loc[:, "정원수":"출근자수"] = minmax.transform(
    X_test[col_na])
display(X_test)

,요일,정원수,휴가자수,출장자수,야근자수,재택근무자수,출근자수,월,년도
일자,,,,,,,,,
2021-01-27,3,0.542614,0.054122,0.418398,0.004789,0.671670,0.664300,1,2021
2021-01-28,4,0.542614,0.067444,0.507418,0.391762,0.652908,0.649451,1,2021
2021-01-29,5,0.542614,0.205662,0.617211,0.000000,0.551595,0.641059,1,2021
2021-02-01,1,0.458807,0.070774,0.335312,0.515326,0.604128,0.584248,2,2021
2021-02-02,2,0.458807,0.032473,0.430267,0.435824,0.589118,0.497095,2,2021
2021-02-03,3,0.458807,0.029975,0.468843,0.004789,0.536585,0.453841,2,2021
2021-02-04,4,0.458807,0.031640,0.504451,0.455939,0.540338,0.441575,2,2021
2021-02-05,5,0.458807,0.121565,0.626113,0.000000,0.480300,0.605552,2,2021
2021-02-08,1,0.458807,0.054122,0.394659,0.660920,0.617261,0.594577,2,2021


In [22]:
voting = blend_models(models, optimize = 'MAE')
voting = tune_model(voting, 
                 optimize = 'MAE', 
                 choose_better = True,
                 n_iter = 30)

voting = finalize_model(voting)
layer1_pred = voting.predict(X_test)
sample['석식계'] = layer1_pred

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,44.5506,3835.2761,61.9296,0.6826,0.1653,0.1130
1,50.2834,4925.5457,70.1822,0.5332,0.1375,0.1019
2,45.4716,3271.9177,57.2007,0.7004,0.1492,0.1136
3,43.3538,3137.7184,56.0153,0.7268,0.1288,0.0986
4,46.4421,4755.1649,68.9577,0.6574,0.1752,0.1174
5,39.2411,2684.5081,51.8122,0.7758,0.1168,0.0880
6,37.3823,2460.0709,49.5991,0.7905,0.1190,0.0883
7,42.2599,2978.2974,54.5738,0.7724,0.1510,0.1085
8,38.4407,2343.0286,48.4048,0.7189,0.1042,0.0845
9,43.1179,3796.0334,61.6120,0.7162,0.1594,0.1085


In [23]:
sample

,일자,중식계,석식계
0,2021-01-27,1015.248763,386.006092
1,2021-01-28,935.156816,395.421187
2,2021-01-29,627.762992,279.572845
3,2021-02-01,1197.140135,544.541336
4,2021-02-02,965.302636,453.645632
5,2021-02-03,922.009973,398.152626
6,2021-02-04,892.967347,450.080575
7,2021-02-05,657.765397,352.422853
8,2021-02-08,1233.564635,631.710606
9,2021-02-09,1023.842213,530.654817


In [25]:
sample.to_csv('sub/Model_Cat_{}.csv'.format(today), index=False)